## I. Practice 'yield'

In [1]:
def practice():
    for i in range(10):
        print('hello')
        yield i

In [2]:
practice()

<generator object practice at 0x111ccd2a0>

In [3]:
list(practice())

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
for q in practice():
    print(q)

hello
0
hello
1
hello
2
hello
3
hello
4
hello
5
hello
6
hello
7
hello
8
hello
9


## II. Implement Scrapy

In [5]:
import scrapy
from scrapy.crawler import CrawlerProcess


class TouristTheSpider(scrapy.Spider):
    name = "Tourist"
    
    
    start_urls = [
        'https://www.tourradar.com/deals/'
        ]
    def parse(self, response):
        links= response.xpath('//div[contains(@class, "con")]//a[@class="pad"]/@href').extract()
        for link in links:
            #crawl the first ten pages only
            for number in range(1, 10, 1):
                base_url = 'https://www.tourradar.com'
                page_url = base_url + link +'?page=' +str(number)
                request = scrapy.Request(page_url, callback=self.parse_page)
                yield request
                
    def parse_page(self, response):

        # Identifying the information we want from each page
        days = response.xpath('//div[contains(@class, "br ")]//dd[contains(@class,"mob")]/text()').extract()
        texts = response.xpath('//div[contains(@class, "bm")]//h4[contains(@itemprop,"name")]/text()').extract()
        #savings = response.xpath('//div[contains(@class, "br ")]//dd[contains(@class,"reg")]/text()').extract()
        prices = response.xpath('//div[contains(@class, "br ")]//span[contains(@class,"prv")]/text()').extract()
        reviews = response.xpath('//div[contains(@class, "bm")]//a[contains(@class, "blank tourLink")]//span/text()').extract()
       
        #there is only one headline per page
        country = response.xpath('//div[contains(@class, "stat")]//h2/text()').extract_first()
                # Yield a dictionary with the values we want.
            
        
        for i in range(len(texts)):
            day = days[i]
            text = texts[i]
            #saving = savings[i]
            price = prices[i]
            review = reviews[i]
            out = {'Headline': text,
                   'Duration': day,
                   'Price': price,
                    #'Savings': saving,
                    'Reviews': review,
                    'Region': country}
            yield out 

process = CrawlerProcess({
'FEED_FORMAT': 'json',
'FEED_URI': 'deals_spider.json',
# Note that because we are doing API queries, the robots.txt file doesn't apply to us.
'ROBOTSTXT_OBEY': True,
'USER_AGENT': 'Inna Munroe\'s Crawler (innafomina43@gmai.com)',
'AUTOTHROTTLE_ENABLED': True,
'HTTPCACHE_ENABLED': True,
'LOG_ENABLED': False,    

})


# Starting the crawler with our spider.
process.crawl(TouristTheSpider)
process.start()
print('Done!')

Done!


In [6]:
import pandas as pd


deals=pd.read_json('deals_spider.json')

In [7]:
#convert it our data to a dataframe
df = pd.DataFrame(deals)

In [8]:
df.shape

(748, 5)

In [9]:
print(df.iloc[0, -1])

331 reviews                        - Excellent


In [10]:
df.head(50)

,Duration,Headline,Price,Region,Reviews
0,12,Europe Escape - 12 Days,"1,680",\n 250+ Europe Special Offers\n,331 reviews - Excellent
1,5,5 Day Skye & Highland Fling,291,\n 250+ Europe Special Offers\n,446 reviews - Excellent
2,3,3 Day Isle of Skye,157,\n 250+ Europe Special Offers\n,650 reviews - Excellent
3,20,European Getaway,"2,159",\n 250+ Europe Special Offers\n,130 reviews - Excellent
4,14,European Wonder,"1,572",\n 250+ Europe Special Offers\n,138 reviews - Excellent
5,14,Europe Jewel - 14 Days,"1,695",\n 250+ Europe Special Offers\n,111 reviews - Excellent
6,3,Skye High,160,\n 250+ Europe Special Offers\n,211 reviews - Excellent
7,8,Croatia Island Hopper from Split - Classic Sta...,570,\n 250+ Europe Special Offers\n,440 reviews - Excellent
8,11,Greek Island Hopper (Fixed Itinerary),"1,194",\n 250+ Europe Special Offers\n,438 reviews - Excellent
9,5,Highland Fling,314,\n 250+ Europe Special Offers\n,214 reviews - Excellent


In [11]:
df.tail(50)

,Duration,Headline,Price,Region,Reviews
698,17,Western Adventure - 17 days,"1,960",\n Page 5 of 250+ Europe Special Offers\n,8 reviews - Good
699,19,European Inspiration (19 Days),"2,473",\n Page 5 of 250+ Europe Special Offers\n,79 reviews - Excellent
700,25,European Escapade (25 Days),"2,313",\n Page 5 of 250+ Europe Special Offers\n,95 reviews - Excellent
701,9,Irish Explorer,949,\n Page 5 of 250+ Europe Special Offers\n,21 reviews - Excellent
702,19,"European Whirl (End Paris, 19 Days)","2,695",\n Page 5 of 250+ Europe Special Offers\n,111 reviews - Excellent
703,10,Iceland Discovery,"2,950",\n Page 5 of 250+ Europe Special Offers\n,20 reviews - Good
704,20,European Experience (20 Days),"1,850",\n Page 5 of 250+ Europe Special Offers\n,88 reviews - Excellent
705,4,Ibiza Blast,388,\n Page 4 of 250+ Europe Special Offers\n,90 reviews - Excellent
706,9,European Magic (9 Days),"1,179",\n Page 4 of 250+ Europe Special Offers\n,85 reviews - Excellent
707,34,Discover Europe,"4,967",\n Page 4 of 250+ Europe Special Offers\n,74 reviews - Excellent
